In [22]:
# %pip install tensorflow
# %pip freeze > requirements.txt

In [23]:
# generate training set
from tensorflow import keras

train_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/training_testing/random-inf-0.8/training/',
	validation_split=0.2,
	subset="training",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)
train_set.class_names


Found 24090 files belonging to 2 classes.
Using 19272 files for training.


['no whale', 'whale present']

In [24]:
# generate validation set
validation_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/training_testing/random-5500-0.8/training/',
	validation_split=0.2,
	subset="validation",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)

Found 11000 files belonging to 2 classes.
Using 2200 files for validation.


In [25]:
for images, labels in train_set:
    preprocessed_images = keras.applications.resnet50.preprocess_input(images)

preprocessed_images, labels


(<tf.Tensor: shape=(8, 224, 224, 3), dtype=float32, numpy=
 array([[[[  -1.939003 ,  -39.779    ,  102.32     ],
          [  -1.939003 ,  -39.779    ,  102.32     ],
          [  -3.939003 ,  -37.779    ,  104.32     ],
          ...,
          [  34.060997 ,   81.221    ,  130.32     ],
          [  39.060997 ,   87.221    ,  130.32     ],
          [  39.060997 ,   87.221    ,  130.32     ]],
 
         [[  -1.939003 ,  -39.779    ,  102.32     ],
          [  -1.939003 ,  -39.779    ,  102.32     ],
          [  -3.939003 ,  -37.779    ,  104.32     ],
          ...,
          [  34.060997 ,   81.221    ,  130.32     ],
          [  39.060997 ,   87.221    ,  130.32     ],
          [  39.060997 ,   87.221    ,  130.32     ]],
 
         [[  -1.939003 ,  -39.779    ,  102.32     ],
          [  -1.939003 ,  -39.779    ,  102.32     ],
          [  -3.939003 ,  -37.779    ,  104.32     ],
          ...,
          [  34.060997 ,   81.221    ,  130.32     ],
          [  39.060997 ,  

In [26]:
# load resnet50 model with pretrained weights
# replace output layer with 2 neurons
# connect to gpu?
# shuffle training set
#  cross entrpoy loss, otimize adam model parameters learning rate 0.0001 epochs 10
# save model
# confusion matrix

In [27]:
# base_model = keras.applications.ResNet50(
# 	weights= None, 
# 	# include_top=False, 
# 	# input_shape= (img_height,img_width,3)
# 	num_classes=2
# 	)

imported_model= keras.applications.ResNet50(include_top=False,
	input_shape=(224,224,3),
	pooling='avg',
	classes=2,
	weights='imagenet'
	# weights=None
)

In [28]:
custom_model = keras.models.Sequential()
custom_model.add(imported_model)
custom_model.add(keras.layers.Dense(2, activation='softmax'))
custom_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_2 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 23538690 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [29]:
from keras.optimizers.legacy import Adam
custom_model.compile(
	optimizer=Adam(learning_rate=0.001),
	loss='sparse_categorical_crossentropy',
	metrics=['accuracy']
)

In [30]:
history = custom_model.fit(
	train_set,
	validation_data=validation_set,
	epochs=10
)

Epoch 1/10
603/603 [==============================] - 1754s 3s/step - loss: 0.2565 - accuracy: 0.8924 - val_loss: 0.2493 - val_accuracy: 0.9105
Epoch 2/10
603/603 [==============================] - 1465s 2s/step - loss: 0.1837 - accuracy: 0.9173 - val_loss: 1.3552 - val_accuracy: 0.5327
Epoch 3/10
603/603 [==============================] - 1473s 2s/step - loss: 0.1716 - accuracy: 0.9245 - val_loss: 0.2261 - val_accuracy: 0.8986
Epoch 4/10
603/603 [==============================] - 1474s 2s/step - loss: 0.1635 - accuracy: 0.9261 - val_loss: 0.8299 - val_accuracy: 0.6891
Epoch 5/10
603/603 [==============================] - 1487s 2s/step - loss: 0.1566 - accuracy: 0.9315 - val_loss: 0.3150 - val_accuracy: 0.8859
Epoch 6/10
603/603 [==============================] - 1587s 3s/step - loss: 0.1514 - accuracy: 0.9355 - val_loss: 0.3116 - val_accuracy: 0.8877
Epoch 7/10
603/603 [==============================] - 1713s 3s/step - loss: 0.1416 - accuracy: 0.9376 - val_loss: 0.4310 - val_accuracy:

In [31]:
# save custom_model
custom_model.save('resnet50_epochs10_samplesinf_seed123.h5')

# 17% charged before 9
# for ten epochos of all the data it takes roughly 5 hours of work